In [41]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("/scratch/ewalt/pdm/rs-uncertainty")
import matplotlib.pyplot as plt
from src.metrics import StratifiedRCU
from src.viz import CloudThresholdVisualizer
import wandb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Configure runs

In [18]:
entity_name = "elwalt"
project_name = "rcu-evaluation"
runs_dir = "results"
run_names = [
    "2023-04-26_11-03-49_cloud_threshold_0-01",
    "2023-04-26_11-03-49_baseline",
    "2023-04-26_11-03-49_cloud_threshold_0-25",
    "2023-04-26_11-03-49_cloud_threshold_0-50",
    "2023-04-26_11-03-47_cloud_threshold_0-75",
    "2023-04-26_11-03-47_cloud_threshold_1-00",
]

In [19]:
api = wandb.Api?

In [20]:
api = wandb.Api()

In [21]:
project = api.project(name="rcu-evaluation")

In [28]:
run = wandb.apis.public.Run(client=api.client, entity=entity_name, project=project_name, run_id="2023-04-26_11-03-47_cloud_threshold_1-00")

ValueError: Could not find run <Run elwalt/rcu-evaluation/2023-04-26_11-03-47_cloud_threshold_1-00 (not found)>

## Load RCUs

In [13]:
rcus = [StratifiedRCU.from_json(os.path.join(runs_dir, rn, "rcu.json")) for rn in run_names]

FileNotFoundError: [Errno 2] No such file or directory: 'results/2023-04-26_11-03-49_cloud_threshold_0-01/rcu.json'

In [14]:
!ls 

/scratch/ewalt/pdm/rs-uncertainty


## WandB CT Runs

In [35]:
api = wandb.Api()
baseline_run = api.runs(
    path="rs-uncertainty/rcu-evaluation",
    filters={"tags": "baseline"}
)[0]
ct_runs = api.runs(
    path="rs-uncertainty/rcu-evaluation",
    filters={"tags": "cloud_threshold"}
)

In [37]:
ct_runs[0].config.keys()

dict_keys(['dataset.name', 'dataset.gt_dir', 'evaluation.tags', 'prediction.name', 'dataset.projects', 'dataset.save_dir', 'evaluation.gt_dir', 'prediction.gt_dir', 'prediction.margin', 'dataset.data_bands', 'dataset.patch_size', 'evaluation.pkl_dir', 'prediction.pkl_dir', 'evaluation.num_bins', 'prediction.projects', 'prediction.save_dir', 'evaluation.data_bands', 'prediction.data_bands', 'prediction.patch_size', 'dataset.split_mask_dir', 'evaluation.projects_east', 'evaluation.projects_west', 'dataset.sampling_strategy', 'evaluation.normalize_mean', 'evaluation.prediction_dir', 'evaluation.projects_north', 'evaluation.variable_names', 'prediction.split_mask_dir', 'dataset.project_shapefiles', 'dataset.s1_reprojected_dir', 'dataset.s2_reprojected_dir', 'prediction.checkpoint_dirs', 'dataset.cloud_prob_threshold', 'evaluation.upsampling_coeffs', 'prediction.sampling_strategy', 'evaluation.normalize_variance', 'prediction.project_shapefiles', 'prediction.s1_reprojected_dir', 'prediction

In [39]:
for ct_run in ct_runs:
    print(ct_run.config["dataset.cloudy_pixels_threshold"], ct_run.name)

0.5 2023-04-25_17-48-18_cloud_threshold_0-50
1 2023-04-25_17-48-18_cloud_threshold_1-00
0.75 2023-04-25_17-48-18_cloud_threshold_0-75
0.25 2023-04-25_17-48-18_cloud_threshold_0-25
0.01 2023-04-25_17-48-18_cloud_threshold_0-01


## ExperimentVisualizer

In [62]:
import pandas as pd

In [83]:
ev = CloudThresholdVisualizer(entity="rs-uncertainty", project="rcu-evaluation")

Found 5 runs for experiment on variable dataset.cloudy_pixels_threshold.


In [84]:
baseline, runs, df = ev.baseline, ev.runs, ev.df

In [85]:
df

,experiment,dataset.cloudy_pixels_threshold,kind,metric,variable,group,value
0,cloud_threshold,0.5,ause,rmse,Cover,9,0.121663
1,cloud_threshold,0.5,agg,rmse,Dens,north,0.126946
2,cloud_threshold,0.5,ause,auce,Cover,959,0.099613
3,cloud_threshold,0.5,agg,mae,MeanH,global,1.457118
4,cloud_threshold,0.5,agg,mse,P95,920,17.013788
...,...,...,...,...,...,...,...
28375,baseline,0.1,agg,ence,Gini,415,0.000599
28376,baseline,0.1,agg,mbe,MeanH,346,0.123375
28377,baseline,0.1,agg,srp,MeanH,477,5.376637
28378,baseline,0.1,ause,auce,MeanH,521,0.043469


In [80]:
baseline.files()[0]

<File artifact/433917062/wandb_manifest.json (application/json) 0.0B>

In [82]:
baseline.log

AttributeError: '<Run rs-uncertainty/rcu-evaluation/oy32thjj (running)>' object has no attribute 'log'

In [86]:
wandb.Api()

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.
